In [ ]:
# Install PyTenNet if running on Colab
try:
    import tensornet
except ImportError:
    !pip install git+https://github.com/tigantic/PyTenNet.git -q
    import tensornet

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tensornet import MPS
from tensornet.algorithms.tebd import tebd, heisenberg_gates, tfim_gates

## 1. TEBD Algorithm Overview

TEBD uses Suzuki-Trotter decomposition to approximate time evolution:

$$|\psi(t)\rangle = e^{-iHt} |\psi(0)\rangle \approx \left(\prod_{\text{odd } j} e^{-ih_{j,j+1}\Delta t} \prod_{\text{even } j} e^{-ih_{j,j+1}\Delta t}\right)^{t/\Delta t} |\psi(0)\rangle$$

Each local gate $e^{-ih_{j,j+1}\Delta t}$ acts on two neighboring sites and can be applied efficiently to an MPS.

## 2. Spin Wave in Heisenberg Chain

We start from a domain wall state $|\uparrow\uparrow\uparrow\downarrow\downarrow\downarrow\rangle$ and watch it spread.

In [ ]:
def create_domain_wall(L: int, wall_pos: int) -> MPS:
    """Create domain wall: |↑↑...↑↓↓...↓⟩ with wall at wall_pos."""
    tensors = []
    for i in range(L):
        t = torch.zeros(1, 2, 1, dtype=torch.complex128)
        if i < wall_pos:
            t[0, 0, 0] = 1.0  # |↑⟩
        else:
            t[0, 1, 0] = 1.0  # |↓⟩
        tensors.append(t)
    mps = MPS(tensors)
    mps.normalize_()
    return mps


def measure_sz(mps: MPS, site: int) -> float:
    """Measure ⟨Sz⟩ at given site."""
    Sz = torch.tensor([[0.5, 0], [0, -0.5]], dtype=mps.tensors[0].dtype)
    return mps.expectation_value_local(Sz, site).real.item()


# System parameters
L = 20
J = 1.0
dt = 0.05
t_max = 5.0
chi_max = 64
num_steps = int(t_max / dt)

# Create initial domain wall
mps = create_domain_wall(L, L // 2)

# Create gates
gates = heisenberg_gates(L=L, J=J, dt=dt)

# Store magnetization profile over time
times = [0.0]
sz_profiles = [[measure_sz(mps, i) for i in range(L)]]

print(f"Simulating domain wall dynamics for t ∈ [0, {t_max}]")
print(f"L={L}, J={J}, dt={dt}, χ_max={chi_max}")

for step in range(num_steps):
    mps = tebd(mps, gates, chi_max=chi_max)
    
    if (step + 1) % 10 == 0:
        t = (step + 1) * dt
        times.append(t)
        sz_profiles.append([measure_sz(mps, i) for i in range(L)])
        print(f"t = {t:.1f}, max χ = {max(t.shape[0] for t in mps.tensors[1:])}")

print("Done!")

In [ ]:
# Plot magnetization dynamics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Heatmap
sz_array = np.array(sz_profiles)
im = ax1.imshow(sz_array.T, aspect='auto', origin='lower', 
                extent=[0, t_max, 0, L], cmap='RdBu_r', vmin=-0.5, vmax=0.5)
ax1.set_xlabel('Time t')
ax1.set_ylabel('Site i')
ax1.set_title('⟨Sz(i,t)⟩ Dynamics')
plt.colorbar(im, ax=ax1, label='⟨Sz⟩')

# Line plots at different times
sites = np.arange(L)
for idx, t in enumerate(times[::len(times)//5]):
    ax2.plot(sites, sz_profiles[idx * len(times)//5], 'o-', 
             label=f't={t:.1f}', markersize=3)
ax2.set_xlabel('Site i')
ax2.set_ylabel('⟨Sz⟩')
ax2.set_title('Magnetization Profiles')
ax2.legend()
ax2.axhline(0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 3. TFIM Quench Dynamics

Simulate a sudden quench in the Transverse-Field Ising Model:
$$H = -J\sum_i Z_i Z_{i+1} - g\sum_i X_i$$

Start from the paramagnetic ground state (large g) and quench to the ferromagnetic phase.

In [ ]:
def create_x_polarized(L: int) -> MPS:
    """Create |+⟩^⊗L state (paramagnetic phase)."""
    tensors = []
    plus = torch.tensor([1.0, 1.0], dtype=torch.complex128) / np.sqrt(2)
    for i in range(L):
        t = torch.zeros(1, 2, 1, dtype=torch.complex128)
        t[0, :, 0] = plus
        tensors.append(t)
    mps = MPS(tensors)
    mps.normalize_()
    return mps


def measure_zz(mps: MPS, i: int, j: int) -> float:
    """Measure ⟨Zi Zj⟩ correlation."""
    Z = torch.tensor([[1.0, 0], [0, -1.0]], dtype=mps.tensors[0].dtype)
    # Contract to get expectation value
    val = mps.expectation_value_two_site(Z, Z, i, j)
    return val.real.item()


# Quench parameters
L = 16
J = 1.0
g_final = 0.5  # Ferromagnetic phase (g < 1)
dt = 0.02
t_max = 4.0
chi_max = 64
num_steps = int(t_max / dt)

# Start from paramagnetic state
mps = create_x_polarized(L)

# Evolve with ferromagnetic Hamiltonian
gates = tfim_gates(L=L, J=J, g=g_final, dt=dt)

# Track order parameter ⟨Z0 ZL/2⟩
times_tfim = [0.0]
order_param = [measure_zz(mps, 0, L//2)]

print(f"TFIM quench: g → {g_final} (ferromagnetic phase)")

for step in range(num_steps):
    mps = tebd(mps, gates, chi_max=chi_max)
    
    if (step + 1) % 5 == 0:
        t = (step + 1) * dt
        times_tfim.append(t)
        order_param.append(measure_zz(mps, 0, L//2))

print("Done!")

In [ ]:
# Plot quench dynamics
plt.figure(figsize=(8, 4))
plt.plot(times_tfim, order_param, 'b-', linewidth=1.5)
plt.xlabel('Time t', fontsize=12)
plt.ylabel('⟨Z₀ Z_{L/2}⟩', fontsize=12)
plt.title('TFIM Quench: Order Parameter Dynamics', fontsize=14)
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Entanglement Growth

Track how entanglement entropy grows during time evolution. For generic states, we expect
linear growth until saturation.

In [ ]:
# Track entanglement entropy during domain wall evolution
L = 16
dt = 0.05
t_max = 3.0
chi_max = 128
num_steps = int(t_max / dt)

mps = create_domain_wall(L, L//2)
gates = heisenberg_gates(L=L, J=1.0, dt=dt)

times_ent = [0.0]
entropies = [mps.entanglement_entropy(L//2 - 1)]
chi_values = [1]

print("Tracking entanglement growth...")

for step in range(num_steps):
    mps = tebd(mps, gates, chi_max=chi_max)
    
    if (step + 1) % 2 == 0:
        t = (step + 1) * dt
        times_ent.append(t)
        entropies.append(mps.entanglement_entropy(L//2 - 1))
        chi_values.append(max(t.shape[0] for t in mps.tensors[1:]))

print("Done!")

In [ ]:
# Plot entanglement growth
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(times_ent, entropies, 'r-', linewidth=1.5)
ax1.set_xlabel('Time t')
ax1.set_ylabel('S(L/2)')
ax1.set_title('Entanglement Entropy Growth')
ax1.grid(True, alpha=0.3)

ax2.plot(times_ent, chi_values, 'g-', linewidth=1.5)
ax2.set_xlabel('Time t')
ax2.set_ylabel('Bond Dimension χ')
ax2.set_title('MPS Bond Dimension Growth')
ax2.axhline(chi_max, color='red', linestyle='--', label=f'χ_max={chi_max}')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

**TEBD Capabilities:**
- Real-time unitary evolution with Trotter decomposition
- Automatic gate generation for Heisenberg and TFIM models
- SVD truncation controls entanglement growth
- Exact for small enough times, approximate for long times

**Key Parameters:**
- `dt`: Trotter time step (smaller = more accurate, slower)
- `chi_max`: Maximum bond dimension (larger = more accurate, more memory)

**When to Use TEBD:**
- Short to medium time dynamics
- Quench dynamics
- Propagating localized excitations
- When entanglement stays bounded